In [36]:
from CARP_solver import *
import cProfile

In [37]:
def single_insert(solu: Solution):
    solu = deepcopy(solu)

    while True:
        remove_route_idx = randint(0, len(solu.routes) - 1)
        l = len(solu[remove_route_idx].tasks)
        if l > 2:
            remove_task_idx = randint(1, l - 2)
            break
    
    task: Task = solu[remove_route_idx][remove_task_idx]
    solu[remove_route_idx].remove_task(idx=remove_task_idx)

    min_cost = MAX_COST
    insert_route_idx, insert_task_idx = None, None
    invert_task_flag = None
    # 遍历整个solution，找出cost最小的插入方式
    for route_idx in range(len(solu.routes)):
        route = solu[route_idx]
        if route.addable(task):
            for task_idx in range(1, len(route.tasks)):
                cost1 = route.insert_cost(task_idx, task)
                cost2 = route.insert_cost(task_idx, task.invert_task)
                if cost1 < min_cost:
                    min_cost = cost1
                    insert_route_idx, insert_task_idx = route_idx, task_idx
                    invert_task_flag = False
                if cost2 < min_cost:
                    min_cost = cost2
                    insert_route_idx, insert_task_idx = route_idx, task_idx
                    invert_task_flag = True
    

    if insert_route_idx is not None:
        solu[insert_route_idx].insert_task(insert_task_idx, task.invert_task if invert_task_flag else task)
    
    solu.calc_cost()
    return solu
    




In [38]:
def double_insertion(solu: Solution):
    solu = deepcopy(solu)

    while True:
        remove_route = choice(solu.routes)
        l = len(remove_route.tasks)
        if l > 3:
            remove_task_idx = randint(1, l - 3)
            break
    

    d_tasks = [remove_route[remove_task_idx], remove_route[remove_task_idx + 1]]
    d_tasks_inv = [d_tasks[1].invert_task, d_tasks[0].invert_task]

    min_cost = remove_route.remove_task(idx=remove_task_idx) + remove_route.remove_task(idx=remove_task_idx) + 100000

    insert_route_idx, insert_task_idx = None, None
    invert_flag = None
    for i in range(len(solu.routes)):
        r = solu[i]
        if r.remain_cap - d_tasks[0].demand - d_tasks[1].demand >= 0:
            for j in range(1, len(r.tasks) - 1):
                cost1 = r.insert_cost(j, d_tasks[0]) + r.insert_cost(j + 1, d_tasks[1])
                cost2 = r.insert_cost(j, d_tasks_inv[0]) + r.insert_cost(j + 1, d_tasks_inv[1])
                if cost1 < min_cost:
                    min_cost = cost1
                    insert_route_idx, insert_task_idx = i, j
                    invert_flag = False
                if cost2 < min_cost:
                    min_cost = cost2
                    insert_route_idx, insert_task_idx = i, j
                    invert_flag = True

    if insert_route_idx is not None:
        if invert_flag:
            solu[insert_route_idx].insert_task(insert_task_idx, d_tasks_inv[0])
            solu[insert_route_idx].insert_task(insert_task_idx + 1, d_tasks_inv[1])
        else:
            solu[insert_route_idx].insert_task(insert_task_idx, d_tasks[0])
            solu[insert_route_idx].insert_task(insert_task_idx + 1, d_tasks[1])
    solu.calc_cost()
    return solu


In [39]:
def main(pop_size, seed, timeout, info):
    gv.init(info)

    random.seed(seed)

    pop = init_pop(pop_size)

    best_eval = min(pop, key=lambda solu: solu.cost).cost
    tabu_list = []
    max_len_tabu = 30

    start = end = perf_counter()
    cnt = 0
    ops = [single_insert, double_insertion]
    
    while end - start < timeout:
        going = [0, 0, 0]
        cnt += 1
        # crossover
        co = sample(pop, k=2)
        new_solu = Solution.crossover(co[0], co[1])
        # new_solu = choice(pop)
        op_type = np.random.rand()
        if op_type < 0.7:
            new_solu = single_insert(new_solu)
            going[0] = 1
        elif op_type < 0.7:
            # new_solu = double_insertion(new_solu)
            going[1] = 1
        else:
            new_solu = merge_split(new_solu)
            going[2] = 1

        
        if new_solu is not None and new_solu not in pop:
            pop = update_pop(pop, new_solu)


        bfs = best_feasible_solu(pop)
        eval = bfs.eval()
        if eval < best_eval:
            best_eval = eval
            print(eval)


        end = perf_counter()
    print(cnt)
    return best_feasible_solu(pop)

In [40]:
# !debug
file_path = "T:\CS311-AI\Proj-CARP\datasets\egl\egl-s1-A.dat"
# timeout = 10
# seed = 10
# !####
info = read_file(file_path)
random.seed(16)
main(30, 1645135, 10, info)
# cProfile.run("print(main(30, random.randint(0, 10000), 90, info))")

7567
7416
7322
7204
7112
7081
7032
6893
6823
6754
6560
6357
6198
6104
6100
6025
5917
5902
5745
5701
778


solution cost:5701, feasible
{'tasks': [(0, 0): 0, (113, 117): 58, (86, 85): 4, (45, 42): 5, (43, 72): 36, (72, 71): 18, (71, 70): 9, (70, 68): 6, (68, 66): 4, (66, 65): 33, (94, 95): 8, (95, 96): 17, (96, 97): 3, (109, 106): 8, (0, 0): 0], 'cost': 808, 'remain_cap': 1}
{'tasks': [(0, 0): 0, (116, 118): 24, (85, 84): 20, (84, 83): 10, (83, 81): 13, (81, 79): 16, (79, 78): 12, (78, 77): 12, (31, 29): 28, (29, 27): 5, (27, 26): 6, (26, 10): 50, (33, 44): 4, (0, 0): 0], 'cost': 899, 'remain_cap': 10}
{'tasks': [(0, 0): 0, (0, 115): 18, (113, 112): 12, (112, 111): 4, (111, 106): 13, (106, 107): 25, (107, 108): 103, (110, 109): 7, (109, 111): 13, (0, 0): 0], 'cost': 464, 'remain_cap': 15}
{'tasks': [(0, 0): 0, (103, 101): 21, (54, 53): 10, (47, 48): 10, (48, 139): 24, (139, 54): 14, (54, 55): 4, (64, 63): 15, (63, 62): 9, (62, 61): 7, (61, 65): 37, (103, 104): 6, (104, 105): 15, (105, 106): 28, (0, 0): 0], 'cost': 698, 'remain_cap': 10}
{'tasks': [(0, 0): 0, (1, 116): 16, (33, 138): 49, (8,